In [19]:
import pandas as pd
import os
import numpy as np

# 1. 读取并展示所有CSV文件的列名
file_paths = ["2015年国内主要城市年度数据.csv", 
              "2016年国内主要城市年度数据.csv", 
              "2017年国内主要城市年度数据.csv"]

# 存储每个文件的列名
all_columns = {}
for file in file_paths:
    # 只读取第一行获取列名
    df = pd.read_csv(file, nrows=0)
    all_columns[file] = list(df.columns)

# 展示所有列名供用户参考
print("发现的CSV文件及其列名:")
for file, columns in all_columns.items():
    print(f"\n文件: {file}")
    print(f"列名: {', '.join(columns)}")


发现的CSV文件及其列名:

文件: 2015年国内主要城市年度数据.csv
列名: 地区, 年份, 国内生产总值, 第一产业增加值, 第二产业增加值, 第三产业增加值, 社会商品零售总额, 货物进出口总额, 年末总人口, 在岗职工平均工资, 普通高等学校在校学生数, 医院、卫生院数, 房地产开发投资额

文件: 2016年国内主要城市年度数据.csv
列名: 地区, 年份, 国内生产总值, 第一产业增加值, 第二产业增加值, 第三产业增加值, 社会商品零售总额, 货物进出口总额, 年末总人口, 在岗职工平均工资, 普通高等学校在校学生数, 医院、卫生院数, 房地产开发投资额

文件: 2017年国内主要城市年度数据.csv
列名: 地区, 年份, 国内生产总值, 第一产业增加值, 第二产业增加值, 第三产业增加值, 社会商品零售总额, 货物进出口总额, 年末总人口, 在岗职工平均工资, 普通高等学校在校学生数, 医院、卫生院数, 房地产开发投资额


In [21]:

dfs = []
for file in file_paths:
    df = pd.read_csv(file)
    # 从文件名提取年份，这里注意原数据已有“年份”列，可校验或直接使用
    # 若文件名年份和数据中“年份”列一致，可直接用数据里的，若需覆盖用下面方式
    year = file.split("年")[0]
    df["年份"] = int(year)  
    dfs.append(df)

# 纵向合并数据
merged_df = pd.concat(dfs, ignore_index=True)

# 2. 处理缺省值，填充为0
merged_df.fillna(0, inplace=True)

# 3. 按照年份聚合，求每年的国内生产总值（对应列名“国内生产总值”）
annual_gdp = merged_df.groupby("年份")["国内生产总值"].sum().reset_index()
print("每年的国内生产总值:")
print(annual_gdp)

每年的国内生产总值:
     年份     国内生产总值
0  2015  277958.37
1  2016  300696.10
2  2017  332562.43


In [23]:

# 4. 计算每个城市2015 - 2017年GDP的年均增长率
# 透视表，行是“地区”（即城市），列是“年份”，值是“国内生产总值”
pivot_df = merged_df.pivot(index="地区", columns="年份", values="国内生产总值")

# 计算年均增长率：[(终值/初值)^(1/年数) - 1] * 100% ，这里年数是2（2015-2017间隔2年）
growth_rates = ((pivot_df[2017] / pivot_df[2015]) ** (1 / 2) - 1) * 100
growth_rates = growth_rates.reset_index(name="年均增长率(%)")

# 找出增长率最高和最低的五个城市
top_five = growth_rates.sort_values("年均增长率(%)", ascending=False).head(5)
bottom_five = growth_rates.sort_values("年均增长率(%)").head(5)

print("\nGDP年均增长率最高的五个城市:")
print(top_five)
print("\nGDP年均增长率最低的五个城市:")
print(bottom_five)


GDP年均增长率最高的五个城市:
    地区   年均增长率(%)
28  西安  13.474172
16  成都  13.398259
24  深圳  13.355030
8   合肥  12.889341
17  拉萨  12.788795

GDP年均增长率最低的五个城市:
      地区   年均增长率(%)
21    沈阳 -10.195788
9   呼和浩特  -5.777610
11    大连  -2.406989
1   乌鲁木齐   2.109128
10   哈尔滨   5.118674


In [25]:
# 5. 对医院、卫生院数进行归一化处理（Min - Max标准化），对应列名“医院、卫生院数”
def min_max_normalize(df, column):
    """对指定列进行Min - Max标准化"""
    return (df[column] - df[column].min()) / (df[column].max() - df[column].min())

# 按年份分组进行归一化
merged_df["医院、卫生院数_normalized"] = merged_df.groupby("年份").apply(
    lambda x: min_max_normalize(x, "医院、卫生院数")).reset_index(0, drop=True)

# 按年份比较各城市医疗资源（医院、卫生院数）的变化
print("\n归一化后的医院、卫生院数(按年份比较):")
normalized_hospitals = merged_df[["年份", "地区", "医院、卫生院数", "医院、卫生院数_normalized"]]
print(normalized_hospitals.sort_values(["年份", "地区"]))


归一化后的医院、卫生院数(按年份比较):
       年份    地区  医院、卫生院数  医院、卫生院数_normalized
9    2015    上海      644            0.388079
35   2015  乌鲁木齐      175            0.077483
32   2015    兰州      164            0.070199
0    2015    北京      701            0.425828
10   2015    南京      213            0.102649
..    ...   ...      ...                 ...
98   2017    重庆      749            0.838372
106  2017    银川       70            0.048837
79   2017    长春      162            0.155814
93   2017    长沙      188            0.186047
90   2017    青岛      410            0.444186

[108 rows x 4 columns]


C:\Users\86152\AppData\Local\Temp\ipykernel_16900\3747699297.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df["医院、卫生院数_normalized"] = merged_df.groupby("年份").apply(


In [27]:
# 6. 提取北京、上海、广州、深圳四个城市2015 - 2017的GDP（国内生产总值）和社会商品零售总额数据
selected_cities = ["北京", "上海", "广州", "深圳"]
selected_data = merged_df[
    merged_df["地区"].isin(selected_cities)
][["年份", "地区", "国内生产总值", "社会商品零售总额"]]

# 重命名列，让结果更直观（可选，也可保持原列名）
selected_data.columns = ["年份", "城市", "GDP", "社会商品零售总额"]

# 保存到新的CSV文件
output_file = "北上广深经济数据.csv"
selected_data.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n已将北京、上海、广州、深圳的数据保存到 {output_file}")


已将北京、上海、广州、深圳的数据保存到 北上广深经济数据.csv
